In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

def read_data(vib):
  dat=pd.read_csv(vib)
  sensorname=dat.keys()[2:-1] 
  return dat, sensorname


def explore(data):
  print('Data overview: ')
  print(data.shape); print()
  print('keys :') ; print(data.keys()); print()
  print( 'status options: ');  print( data['machine_status'].unique()); print()
  print (data['machine_status'].value_counts()); print()
  info=data.describe()
  varianz=pd.DataFrame({'var':data.var()})
  info=pd.concat([info,varianz.transpose()])
  return data.head(), data.tail(), info


def manipulate_X(data, printplot=False):
    data=data.drop(labels=['sensor_15'],axis=1)
    data=data.drop(labels=['sensor_00'],axis=1)

    data['sensor_51'][110000:140000]=data['sensor_50'][110000:140000] 
    data=data.drop(labels=['sensor_50'],axis=1)


    data=data.drop(labels=['sensor_06','sensor_07','sensor_08','sensor_09'],axis=1)
    data=data.fillna(method="pad",limit=30)
    data=data.dropna()
    if printplot==True:
        print((data.isna().sum()))
        plotting_stuff((data.isna().sum()[2:-1]),'bar','fill_nan',saving=True)
        
    return data

def preprocess(data):
    from sklearn import preprocessing

    le = preprocessing.LabelEncoder()
    le.fit(data)
    encoded_y=le.transform(data)
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print(le_name_mapping)
    return pd.DataFrame(encoded_y,columns=['target'])

#call series_to_supervise() to create shifted data.
#

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, namen = list(),list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        namen +=[('sensor%d(t-%d)' %(j+1, i)) for j in range (n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            namen +=[('sensor%d(t)' %(j+1)) for j in range (n_vars)]
        else:
            namen +=[('sensor%d(t+%d)' '%'(j+1, i)) for j in range (n_vars)]
    agg = pd.concat(cols, axis=1)
    agg.columns=namen

    if dropnan:
        agg.dropna(inplace=True)
    return agg
# splitting Data into Train Validation Test Sets
def splitting_and_shape_data(data_x,data_y):    
    train_X=data_x[0:120000].values
    train_Y=data_y[0:120000].values
    
    val_X=data_x[140000::].values
    val_Y=data_y[140000::].values
    
    test_X=data_x[120000:140000].values
    test_Y=data_y[120000:140000].values
      
    train_X.astype('float32')
    val_X.astype('float32')
    test_X.astype('float32')
    
    return train_X,train_Y,val_X,val_Y,test_X,test_Y,    
#creating perticular 3 shape for LSTM sample timestap and dataset features
def reshape_for_Lstm(data):    
    timesteps=1
    samples=int(np.floor(data.shape[0]/timesteps))

    data=data.reshape((samples,timesteps,data.shape[1]))       
    return data

#interpreting data class correctly
def one_hot(train_Y,val_Y,test_Y):    
    from sklearn.preprocessing import OneHotEncoder
    
    oneHot=OneHotEncoder()
    oneHot.fit(train_Y.reshape(-1,1))
    
    train_Y_Hot=oneHot.transform(train_Y.reshape(-1,1)).toarray()
    val_Y_Hot  =oneHot.transform(val_Y.reshape(-1,1)).toarray()
    test_Y_Hot =oneHot.transform(test_Y.reshape(-1,1)).toarray()
    
    return train_Y_Hot,val_Y_Hot,test_Y_Hot

# using LSTM model neuron 75 activation fn relu used for probablity distribution
#of classification
def model_setup_seq(in_shape):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM
    from tensorflow.keras.layers import Dropout
    from tensorflow.keras.layers import Dense
    
    model = Sequential()
    model.add(LSTM(75,activation='relu', input_shape=(in_shape[1],in_shape[2]), 
                   return_sequences=True)  )

    model.add(LSTM(56,activation='relu')) # 56 layer used
    model.add(Dense(1)) # for 1 neuron prediction
    
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
#softmax to normalize output over probablity distribution
#training the model    
def model_setup_Fapi(in_shape):
    from tensorflow.keras.layers import LSTM
    from tensorflow.keras.layers import Dropout
    from tensorflow.keras.layers import Dense
    
    inputs= tf.keras.Input(shape=(in_shape[1],in_shape[2]))
    x=LSTM(2048,activation='relu', input_shape=(in_shape[1],in_shape[2]),return_sequences=True)(inputs)
    x=LSTM(2048,activation='relu')(x)
    out_signal=Dense(1, name='signal_out')(x)
    out_class=Dense(3,activation='softmax', name='class_out')(x)
    
    model=tf.keras.Model(inputs=inputs, outputs=[out_signal,out_class])
    
    model.compile(loss={'signal_out':'mean_squared_error',
                        'class_out' :'categorical_crossentropy'},
                         optimizer='adam',
                         metrics={'class_out':'acc'})
    
    print(model.summary())
    return model

def plot_training(history,what='loss',saving=False,name='training'):
    fig=plt.figure()
    plt.plot(history[0])
    plt.plot(history[1])
    plt.xlabel('epoch')
    plt.legend(['train', 'test'])
    if what=='loss':
        plt.title('model loss')
        plt.ylabel('loss')
    elif what=='acc':   
        plt.title('model Acc')
        plt.ylabel('Accuracy')   
    if saving==True:
        fig.savefig( name +'_'+ what + '.png', format='png', dpi=300, transparent=True)


    plt.xlabel('epoch')
    plt.legend(['train', 'test'])
    if saving==True:
        fig.savefig( name +'_ACC.png', format='png', dpi=300, transparent=True)  
    plt.show()
    
def plot_signal_hat(Y_hat,Y_test,saving=False,name='results_signal'):
    fig= plt.figure()
    plt.plot(Y_hat)
    plt.plot(Y_test)
    plt.legend(['target','target_predicted'])
    plt.ylabel('Zustand')
    plt.title('Pediction on test data')
    if saving==True:
        fig.savefig( name +'.png', format='png', dpi=300, transparent=True)
    plt.show()
        
def plot_class_hat(Y_hat,Y_test,saving=False,name='results_class'):   
    import matplotlib.pyplot as plt
    from matplotlib import cm
    import numpy as np
    x=np.linspace(1,len(Y_hat),len(Y_hat))

    fig=plt.figure()
    ax1=fig.add_subplot(111)
    ax1=plt.plot(x,Y_test)
    ax1=plt.scatter(x,Y_hat,c=cm.hot(np.abs(Y_hat)), edgecolor='none') 
    plt.legend(['target','target_predicted'])
    if saving==True:
        fig.savefig( name +'.png', format='png', dpi=300, transparent=True)
    plt.show()
       
   
if __name__ == '__main__':
    
   
    data,sensorname=read_data('/content/drive/MyDrive/vib.csv')

 
    encoded_y=preprocess(data['machine_status'])
    Values=pd.concat([data[sensorname],encoded_y],axis=1)

 
    Values=manipulate_X(Values, printplot=False); sensorname=Values.keys()[:-1] 


    Future=1

    data_win=series_to_supervised(Values, n_in=Future, n_out=1)
    to_remove_list =['sensor'+str(n)+'(t)' for n in range(1,len(Values.columns)+1)] #now remove all non shifted elements again. so we retreive elements and shifted target

    data_y=data_win.iloc[:,-1] 
    data_x=data_win.drop(to_remove_list, axis=1) 
    data_x.drop(data_x.columns[len(data_x.columns)-1], axis=1, inplace=True)
    

    
    train_X,train_Y,val_X,val_Y,test_X,test_Y=splitting_and_shape_data(data_x,data_y)
    train_Y_Hot,val_Y_Hot,test_Y_Hot=one_hot(train_Y,val_Y,test_Y)

    scaler=MinMaxScaler().fit(train_X)
    train_X=scaler.transform(train_X) 
   
    scaler=MinMaxScaler().fit(val_X)
    val_X=scaler.transform(val_X)  
    
    scaler=MinMaxScaler().fit(test_X)
    test_X=scaler.transform(test_X)  

    train_X=reshape_for_Lstm(train_X)
    val_X=reshape_for_Lstm(val_X)
    test_X=reshape_for_Lstm(test_X)

    
    Train=True
    inputshape_X=(train_X.shape)
  
    
    if Train==True:

    #epocs defined here
        model=model_setup_Fapi(inputshape_X)
        history = model.fit(train_X, [train_Y, train_Y_Hot], epochs=400, batch_size=2048, validation_data=(val_X, [val_Y,val_Y_Hot]), shuffle=False)
        plot_training([history.history['class_out_loss'],history.history['val_class_out_loss']],
                      what='loss',
                      saving=True,
                      name=('training_'+ str(Future)))  
        plot_training([history.history['class_out_acc'],history.history['val_class_out_acc']],
                      what='acc',
                      saving=True,
                      name=('training_'+ str(Future))) 
        model.save('./model/Pump_LSTM_Fapi_4_'+ str(Future))
 
    else:  
        model=tf.keras.models.load_model('./model/Pump_LSTM_Fapi_4_1')

    [yhat,yclass] = model.predict(test_X)
    print(yhat)
    Yclass=[np.argmax(yclass[i],0) for i in range(len(yclass))] # get final class
    Y_HOT = [np.argmax(test_Y_Hot[i],0) for i in range(len(test_Y_Hot))] 
    plot_signal_hat(test_Y, yhat,saving=True, name='Prediction_Signal_fapi3_42_'+ str(Future))
    plot_signal_hat(Yclass, Y_HOT ,saving=True, name='Prediction_class_fapi3_42_'+ str(Future))




  
    

    

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section

# New Section